# This is DEV test notebook, it has little useful.

In [8]:
# module reloading script, fetches updates to ml4fmri package
import sys, importlib
from pathlib import Path

PKG = "ml4fmri"
# adjust as needed; this assumes your repo root is the CWD and sources are in ../src
SRC = Path().joinpath("../src").resolve()

def _ensure_src_on_path():
    p = str(SRC)
    if p not in sys.path:
        sys.path.insert(0, p)

def fresh_import_ml4fmri():
    """
    Remove any cached ml4fmri modules and import from SRC.
    Returns the ml4fmri module and (if available) meanMLP class.
    """
    _ensure_src_on_path()

    # purge previous imports so code changes take effect
    to_delete = [k for k in list(sys.modules) if k == PKG or k.startswith(PKG + ".")]
    for k in to_delete:
        del sys.modules[k]

    importlib.invalidate_caches()

    ml4fmri = __import__(PKG)
    meanMLP = None
    try:
        from ml4fmri.models import meanMLP 
    except Exception as e:
        print("Note: could not import meanMLP yet:", e)

    try:
        import inspect
        print("ml4fmri imported from:", Path(inspect.getfile(ml4fmri)).parent)
    except Exception:
        pass
    print("ml4fmri ready")
    return ml4fmri

# --- first import (or manual reload later) ---
ml4fmri = fresh_import_ml4fmri()

# get sample data for experiments
from abide import load_data as load_abide
from cobre import load_data as load_cobre

data, labels = load_abide()
data.shape, labels.shape

data, labels = data[:100], labels[:100]

In [25]:
from abide import load_data as load_abide
from cobre import load_data as load_cobre

data, labels = load_abide()
DATA, LABELS = data, labels
data.shape, labels.shape

# data, labels = data[:100], labels[:100]

# test cvbench

In [10]:
# ml4fmri = fresh_import_ml4fmri()
from ml4fmri import cvbench # runs CV experiments with implemented models on the given data
# report = cvbench(data, labels, n_folds=10)
report = cvbench(data, labels, ['LR', 'meanMLP'], n_folds=10)
report.plot_scores()

In [11]:
report.plot_scores_h(metric='test_f1_macro')

In [12]:
# extract report dataframes

test_df = report.get_test_dataframe() # returns dataframe with test metrics: 
# classification metrics and loss on the test data and training time per fold for each model.
# Useful for model comparison.

train_df = report.get_train_dataframe() # returns dataframe with training logs: 
# losses and classification metrics on the training and validation data per fold for each model. 
# Useful for training inspection.

meta_df = report.get_meta() # dictionary with metadata about the training process:
# ['models', 'n_folds', 'val_ratio', 'random_state', 'input_size', 'n_classes', 'train_indices', 'val_indices', 'test_indices']
# the last 3 give you CV indices per fold, so that you can potentially replicate the splits in your experiments.

In [14]:
report.plot_training_curves()

In [15]:
train_df = report.get_train_dataframe()
test_df = report.get_test_dataframe()
meta = report.meta

In [ ]:
report.save() # saves the report to disk: 
# cvbench_train.csv with train dataframe, 
# cvbench_test.csv with test dataframe,
# cvbench_meta.json with metadata,